In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import joblib

In [2]:
df = pd.read_csv(r"E:\APythonSoftware_2025\MP4\swiggy.csv")
df.head()

,id,name,city,rating,rating_count,cost,cuisine,lic_no,link,address,menu
0,567335,AB FOODS POINT,Abohar,--,Too Few Ratings,₹ 200,"Beverages,Pizzas",22122652000138,https://www.swiggy.com/restaurants/ab-foods-po...,"AB FOODS POINT, NEAR RISHI NARANG DENTAL CLINI...",Menu/567335.json
1,531342,Janta Sweet House,Abohar,4.4,50+ ratings,₹ 200,"Sweets,Bakery",12117201000112,https://www.swiggy.com/restaurants/janta-sweet...,"Janta Sweet House, Bazar No.9, Circullar Road,...",Menu/531342.json
2,158203,theka coffee desi,Abohar,3.8,100+ ratings,₹ 100,Beverages,22121652000190,https://www.swiggy.com/restaurants/theka-coffe...,"theka coffee desi, sahtiya sadan road city",Menu/158203.json
3,187912,Singh Hut,Abohar,3.7,20+ ratings,₹ 250,"Fast Food,Indian",22119652000167,https://www.swiggy.com/restaurants/singh-hut-n...,"Singh Hut, CIRCULAR ROAD NEAR NEHRU PARK ABOHAR",Menu/187912.json
4,543530,GRILL MASTERS,Abohar,--,Too Few Ratings,₹ 250,"Italian-American,Fast Food",12122201000053,https://www.swiggy.com/restaurants/grill-maste...,"GRILL MASTERS, ADA Heights, Abohar - Hanumanga...",Menu/543530.json


In [3]:
df["rating"] = df["rating"].replace("--",np.nan)
df['rating'] = pd.to_numeric(df['rating'], errors='coerce')

In [4]:
df.dropna(inplace=True)

In [5]:
df.drop(df[df["cuisine"] == "8:15 To 11:30 Pm"].index, inplace=True)

In [6]:
df.shape

(61343, 11)

In [7]:
df["rating_count"] = df["rating_count"].replace('Too Few Ratings','12')

df["rating_count"] = df["rating_count"].str.replace("+ ratings","",regex=False)

df["rating_count"] = df["rating_count"].str.replace("K","000",regex=False)

df["rating_count"] = pd.to_numeric(df["rating_count"], errors='coerce') #--> Converting the datatype

In [8]:
df.drop(df[df["rating_count"] < 21 ].index, inplace=True)

In [9]:
df.shape

(39739, 11)

In [10]:
df['rating_count'].value_counts()

rating_count
100      20514
50       11985
500       4390
1000      2737
5000        98
10000       15
Name: count, dtype: int64

In [11]:
df[df.duplicated()]

,id,name,city,rating,rating_count,cost,cuisine,lic_no,link,address,menu


In [12]:
df.isnull().sum()

id              0
name            0
city            0
rating          0
rating_count    0
cost            0
cuisine         0
lic_no          0
link            0
address         0
menu            0
dtype: int64

In [13]:
df.drop(["id","lic_no","link","menu","address"],axis="columns",inplace=True)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 39739 entries, 1 to 148509
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          39739 non-null  object 
 1   city          39739 non-null  object 
 2   rating        39739 non-null  float64
 3   rating_count  39739 non-null  int64  
 4   cost          39739 non-null  object 
 5   cuisine       39739 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 2.1+ MB


In [15]:
df.dropna(subset = ["cost"],inplace = True) #--> Removing the Null values

df["cost"] = df["cost"].str.replace("₹", "", regex=False)

df["cost"] = pd.to_numeric(df["cost"], errors="coerce") #--> Converting the datatype

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 39739 entries, 1 to 148509
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          39739 non-null  object 
 1   city          39739 non-null  object 
 2   rating        39739 non-null  float64
 3   rating_count  39739 non-null  int64  
 4   cost          39739 non-null  int64  
 5   cuisine       39739 non-null  object 
dtypes: float64(1), int64(2), object(3)
memory usage: 2.1+ MB


In [17]:
# Function to extract value after the last comma inside quotes
def extract_city_name(city):
    if pd.isna(city):
        return city  # Skip NaNs
    city = city.strip()

    # Remove outer quotes if present
    if (city.startswith('"') and city.endswith('"')) or (city.startswith("'") and city.endswith("'")):
        city = city[1:-1]

    # Extract part after the last comma, or return as is
    if ',' in city:
        return city.split(',')[-1].strip()
    else:
        return city.strip()


In [18]:
# Apply function to the 'city' column
df['city_cleaned'] = df['city'].apply(extract_city_name)

# Check the result
print(df['city_cleaned'].unique())

['Abohar' 'Adilabad' 'Adityapur' 'Adoni' 'Agartala' 'Agra' 'Ahmedabad'
 'Ahmednagar' 'Ajmer' 'Akola' 'Alappuzha' 'Aligarh' 'Alipurduar'
 'Allahabad' 'Alwar' 'Ambala' 'Ambikapur' 'Ambur' 'Amravati' 'Amreli'
 'Amritsar' 'Anand' 'Anantapur' 'Ankleshwar' 'Arambagh' 'Arrah' 'Asansol'
 'Aurangabad' 'Aurangabad_bihar' 'Azamgarh' 'Baddi' 'Bagalkot' 'Bagdogra'
 'Bahadurgarh' 'Bahraich' 'Balangir' 'Balasore' 'Ballari' 'Balrampur'
 'Balurghat' 'Banda' 'Bangalore' 'Bantwal' 'Bapatlachirala' 'Baramati'
 'Baran' 'Bardhaman' 'Bardoli' 'Bareilly' 'Barmer' 'Barnala' 'Barshi'
 'Barwani' 'Basirhat' 'Basti' 'Batala' 'Bathinda' 'Beawar' 'Beed'
 'Begusarai' 'Bela-pratapgarh' 'Belgaum' 'Berhampore' 'Berhampur'
 'Bettiah' 'Betul' 'Bhadohi' 'Bhadrachalam' 'Bhagalpur' 'Bhandara'
 'Bharabanki' 'Bharatpur' 'Bharuch' 'Bhatkal' 'Bhavnagar' 'Bhawanipatna'
 'Bhilai' 'Bhilwara' 'Bhimavaram' 'Bhind' 'Bhiwadi' 'Bhiwani' 'Bhopal'
 'Bhubaneswar' 'Bhuj' 'Bhusawal' 'Bidar' 'Biharsharif' 'Bijapur' 'Bijnor'
 'Bikaner' 'Bilasp

In [19]:
df.to_csv("cleaned_data.csv",index=False)

In [21]:
# Data encoding 
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

# Read your CSV file
cleaned_df = pd.read_csv(r"E:\APythonSoftware_2025\MP4\cleaned_data.csv")

label = LabelEncoder()
encoded_name = label.fit_transform(cleaned_df["name"])

# One-hot encode 'main_city' and 'sub_city'
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=True)
encoded_cities = encoder.fit_transform(cleaned_df[["city","cuisine"]])

encoded_df = pd.DataFrame.sparse.from_spmatrix(
    encoded_cities,
    columns=encoder.get_feature_names_out(["city","cuisine"]),
    index=cleaned_df.index
)

numeric_df = cleaned_df.drop(columns=["city","cuisine","name"])
mixed_df = pd.concat([numeric_df, encoded_df],axis=1)


mixed_df.to_csv("encoded_data.csv", index=False)

In [22]:
# Load cleaned data
df = pd.read_csv("cleaned_data.csv")

name_encoder = LabelEncoder()
df['name_encoded'] = name_encoder.fit_transform(df['name'])

with open("encoders.pkl", "wb") as f:
    pickle.dump({"Encoder": encoder, "Label": name_encoder}, f)


In [23]:
encoded_df = pd.read_csv("encoded_data.csv")
encoded_df['name_encoded'] = df['name_encoded']

# Save updated version
encoded_df.to_csv("name_encoded_data.csv", index=False)

In [24]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [25]:
# Label-encoded data (contains name_encoded and numeric features)
name_encoded_df = pd.read_csv("name_encoded_data.csv")

# One-hot encoded data for city and cuisine
encoded_df = pd.read_csv("encoded_data.csv")  # previously saved from sparse one-hot encoding

In [ ]:
assert name_encoded_df.shape[0] == encoded_df.shape[0]

In [27]:
final_df = pd.concat([name_encoded_df[["name_encoded", "rating", "rating_count", "cost"]],
                      encoded_df.drop(columns=["rating", "rating_count", "cost"])], axis=1)

In [ ]:
final_df.drop(cloumns=["city_cleaned"])

In [28]:
final_df.to_csv("encoded_data_with_name.csv", index=False)

In [34]:
final_df.head()

,name_encoded,rating,rating_count,cost,city_cleaned,"city_Abids & Koti,Hyderabad",city_Abohar,"city_Adajan,Surat",city_Adilabad,city_Adityapur,...,"cuisine_Turkish,Arabian","cuisine_Turkish,Fast Food","cuisine_Turkish,Lebanese","cuisine_Turkish,Mediterranean","cuisine_Turkish,Middle Eastern","cuisine_Vietnamese,Asian","cuisine_Waffle,Bakery","cuisine_Waffle,Beverages","cuisine_Waffle,Desserts","cuisine_Waffle,Ice Cream"
0,10938,4.4,50,200,Abohar,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,25801,3.8,100,100,Abohar,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,25775,4.0,100,150,Abohar,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2921,4.4,50,300,Abohar,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,15093,4.1,100,150,Abohar,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
df = pd.read_csv("encoded_data_with_name.csv")

In [38]:
# Identify columns with datatype 'object'
object_cols = df.select_dtypes(include=['object']).columns

# Display the column names
print("Columns with object datatype:", object_cols)


Columns with object datatype: Index(['city_cleaned'], dtype='object')


In [40]:
X = df.drop(columns=['city_cleaned'])
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


kmeans = KMeans(n_clusters=4, random_state=42)
clusters = kmeans.fit_predict(X_scaled)

In [41]:
df['cluster'] = clusters
df.to_csv("clustered_data.csv", index=False)